In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 100% !important; }</style>"))

In [76]:
#directed unweighted graph (only connections, i.e. A->B )
class DirectedUnweightedGraph():
    def __init__(self):
        self.edges = {}
        self.nodeCount = 0
        self.nodeWidth = 1
 
    def addEdge(self,fromNode,toNode):
        self.nodeWidth = max( self.nodeWidth,len(str(fromNode)),len(str(toNode)) )
        if fromNode not in self.edges:
            self.edges[fromNode] = []
            self.nodeCount += 1
        if toNode not in self.edges:
            self.nodeCount += 1
            self.edges[toNode] = []
        if toNode not in self.edges[fromNode]:
            self.edges[fromNode].append(toNode)
        
    def printMatrix(self):
        orderList = [0]*self.nodeCount
        orderNo = 0
        for node in sorted(self.edges.keys()):
            orderList[orderNo] = node
            orderNo += 1

        print(' '*self.nodeWidth+' ',' '.join([str(node).rjust(self.nodeWidth) for node in sorted(self.edges.keys())]) )
        
        for fromNode in sorted(self.edges.keys()):
            rowStr = str(fromNode).rjust(self.nodeWidth)+")"
            for colNo in range(orderNo):
                toNode = orderList[colNo]
                if toNode in self.edges[fromNode]:
                    rowStr += ' '*self.nodeWidth+'x'
                else:
                    rowStr += ' '*self.nodeWidth+'.'
            print(rowStr)
 
    def hasCycleHelper(self, srcNode, nodesVisited, nodesChecked):
        nodesVisited[srcNode] = True
        nodesChecked[srcNode] = True
        for connectedNode in self.edges[srcNode]:
            if not nodesVisited[connectedNode]:
                if self.hasCycleHelper(connectedNode, nodesVisited, nodesChecked):
                    return True
            elif nodesChecked[connectedNode]:
                return True
 
        nodesChecked[srcNode] = False
        return False
 
    # Returns true if graph is cyclic else false
    def hasCycle(self):
        nodesVisited = {}
        nodesChecked = {}
        for node in self.edges.keys():
            nodesVisited[node] = False
            nodesChecked[node] = False
        
        for node in self.edges.keys():
            if not nodesVisited[node]:
                if self.hasCycleHelper(node,nodesVisited,nodesChecked):
                    return True
        return False

In [79]:
myG = DirectedUnweightedGraph()
myG.addEdge('A','B')
myG.addEdge('A','C')
myG.addEdge('A','DDD')
myG.addEdge('B','C')
myG.addEdge('B','D')
myG.addEdge('C','D')
myG.addEdge('D','FF')

In [81]:
myG.printMatrix()
myG.hasCycle()

       A   B   C   D DDD  FF
  A)   .   x   x   .   x   .
  B)   .   .   x   x   .   .
  C)   .   .   .   x   .   .
  D)   .   .   .   .   .   x
DDD)   .   .   .   .   .   .
 FF)   .   .   .   .   .   .


False

In [230]:
""" get transport from node A to B """
""" allows multiply counting bottleneckes, so traversing an edge does not reduce its capacity """
""" this one has 3 lines of cool comments """
debugTransport = False

def adjacentNodes(path, srcNode):
    connectedToSourceNodes = []
    for adjacentNode in range(len(path)):
        if path[srcNode][adjacentNode] > 0:
            connectedToSourceNodes.append(adjacentNode)
    return connectedToSourceNodes
    
def getTransportHelper(path, curNode, tgtNode, solvedNodes, maxTransportVals, visitedNodes, depth):
    connectedNodes = adjacentNodes(path, curNode)
    visitedNodes[curNode] = True
    #throughput from curNode to target is sum of each connected/child node's throughput, limited by the amount we can transfer directly to the children (path[cur][child])
    toTarget = 0
    for adjNode in connectedNodes:       
        if adjNode == tgtNode:
            adjTransport = 99999999
        elif solvedNodes[adjNode]:
            adjTransport = maxTransportVals[adjNode]
        elif visitedNodes[adjNode]:
            continue
        else:
            adjTransport = getTransportHelper(path, adjNode, tgtNode, solvedNodes, maxTransportVals, visitedNodes, depth+1)
        toTarget += min(path[curNode][adjNode] , adjTransport)
    
    maxTransportVals[curNode] = max(maxTransportVals[curNode], toTarget)  #shouldn't have toTarget greater...
    solvedNodes[curNode] = True
    return toTarget


def getTransportTo(path, tgtNode):
    N = len(path)
    maxTransportVals = [0] * N
    solvedNodes = [False] * N
    for srcNode in range(N):
        if srcNode != tgtNode:
            visitedNodes = [False] * N
            getTransportHelper(path, srcNode, tgtNode, solvedNodes, maxTransportVals, visitedNodes,0)
    
    return maxTransportVals

def answer(entrances, exits, path):
    tot = 0
    for tgt in exits:
        maxTransport = getTransportTo(path, tgt)
        for src in entrances:
            tot += maxTransport[src]
    return tot

In [ ]:
#used by calculateMinCostMat to get the distance of all nodes from a single, given node
#useful in general to find shortest path between nodes in a complete graph
def calcualteCostToNodesFromSource(costMat, sourceNode):
    #defining a function inside a function. this is local to the outer function (calcualteCostToNodesFromSource)
    #just returns next True element number from workList, or -1 if no True elements
    def getNextWorkNode(workList, N):
        for nodeNo in range(N):
            if workList[nodeNo]:
                return nodeNo  #return first True element
        return -1  # no work left to be done (all False)    N = len(costMat)  #count of nodes including start/end

    N = len(costMat)
    nodeList = [k for k in range(N) if k != sourceNode] #in path from beginNode to endNode, we don't need to re-visit beginNode
    needsWork = [False]*N  #every time we update cost-to-get-to-a-node, set node# element True. We're done when all false
    nodeCosts = [999]*N    #minimum cost-to-get-to-a-node that we know of for each node. Initially all are infinity
    nodePaths = [[] for k in range(N)] #we save minimum cost path-to-reach-node here. We do need to know the paths as well as costs
    needsWork[sourceNode] = True  #start by branching away from the source/begin node
    nodeCosts[sourceNode] = 0     #used on first step to set initial cost=0
    #prevNodes = [0 for k in range(N)]  #could just store the from/previous node from best path to that node. If want to get the full path just step backwards to beginNode. More work to unravel the path but this way is mroe efficient. Efficiency not really a problem at the moment, though

    while True:
        fromNode = getNextWorkNode(needsWork, N)  #get next node that needs work (first True element of needsWork)
        if fromNode == -1:  #no work left, we're done
            break

        #print(fromNode)
        curPath = nodePaths[fromNode]      # path taken (list) to reach current node (excluding beginNode). It will include fromNode
        curPathCost = nodeCosts[fromNode]  # cost/distance from following this path     
        newPathLen = len(curPath) + 1    #might use later if we prefer to keep the longest or shortest path when equal cost...

        nodesToVisit = (node for node in nodeList if node not in curPath)  #i think this is roughly as efficient as putting the if inside the loop? generates an iterator
        for toNode in nodesToVisit:  #branch from our fromNode to each other nodes (toNode's)
            edgeCost = costMat[fromNode][toNode]  #get the given cost to travel directly from fromNode to toNode
            newPathCost = edgeCost + curPathCost  #unnecessary intermediate step for clarity
            toNodeCost = nodeCosts[toNode]
            if newPathCost < toNodeCost:
                nodeCosts[toNode] = newPathCost         #new minimum cost to reach toNode
                nodePaths[toNode] = curPath + [toNode]  #update best path to toNode with currentPath. Append toNode
                #  prevNodes[toNode] = fromNode            #save just the previous node in our path?
                needsWork[toNode] = True                #indicate we need to propagate/branch from toNode given new lower cost to reach

            #maybe not needed. handling to prefer longer or shorter paths when cost is equal. If keeping, make function for copied update-toNode-stuff
            elif newPathCost == nodeCosts[toNode]:
                #print('doin the equals thing:',fromNode, 'to', toNode)
                toPathLen = len(nodePaths[toNode])
                if newPathLen > toPathLen: 
                    nodeCosts[toNode] = newPathCost         #new minimum cost to reach toNode
                    nodePaths[toNode] = curPath + [toNode]  #update best path to toNode with currentPath. Append toNode
                    #prevNodes[toNode] = fromNode            #save just the previous node in our path?
                    needsWork[toNode] = True                #This is only needed to update the path in this version where we save the whole path. The cost was not reduced, we just want to propagatethe path if continued elsewhere

        needsWork[fromNode] = False   #having finished this iteration of while loop, fromNode o longer requires branching/work until/unless it gets updated again
    
    #return nodeCosts
    return nodeCosts, nodePaths


#given an NxN matrix representing a (complete, bidirected?) graph of costs to travel directly between any 2 of N nodes (edge length/cost),
#recalcuate the costs as the minimum cost path rather than the direct cost (minimum over multipel edges if short than connecting edge)
#e.g. if it's cheaper to go node A-to-B as A->C->B than as A->B (=costMat[A][B]), we set the value at minCostMat[A][B] = A->C->B (=costMat[A][B] + costMat[B][C]) 
#i guess it doesn't matter but the convention here is that the row is the from-node and the column is the to-node costMat[FROM][TO] = cost from FROM to TO
#negative edge costs are allowed. We don't revisit edges, so any negative loops won't be detected or result in -∞  cost
def calculateMinCostMat(costMat):
    minCostMat = []
    minPathMat = []
    for rowNo in range(len(costMat)):
        minCostRow,minPaths = calcualteCostToNodesFromSource(costMat, rowNo)
        minCostMat.append(minCostRow)
        minPathMat.append(minPaths)
    
    return minCostMat, minPathMat


#This checks for inifite loops. If a circuit has negative cost to traverse, one can traverse it infinitely. must use calculateMinCostMat(costMat) first
# and keep decreasing cost (for time limit problem, you can follow the loop to gain infinite time, then travel anywhere without restriction)
def hasNegativeInfinityLoop(costMat):
    N = len(costMat)
    for rowNo in range(N):
        for colNo in range(N):
            if costMat[rowNo][colNo] + costMat[colNo][rowNo] < 0:  #shortest path from A to B plus B to A, a loop.
                return True, (rowNo,colNo)
    
    return False, ()

In [ ]:
def transportTo(path,)

def transportHelper(path, fromNode, visitedNodes, childrenCache, distanceCache):
    visitedNodes[fromNode] = True
    
    connectedNodes = []
    for toNode in range(N):
        if path[fromNode][toNode] > 0:
            connectedNodes.append(toNode)
    
    maxCapToTarget[target] = 0
    maxCapToTarget[target] = sum(maxCapToTarget
    for toNode in connectedNodes:
        if not visitedNodes[toNode]:
            

def totalTransport(path):
    output = deepcopy(path)
    N = len(path)
    visitedNodes = [False]*N
    childrenCache = [False]*N
        
    for fromNode in range(N):
        visitedNodes = [False]*N
        transportHelper(path, fromNode, visitedNodes, childrenCache, distanceCache)


In [ ]:
def getAdjacentNodes(path, srcNode):
    connectedToSourceNodes = []
    for adjacentNode in range(len(path)):
        if path[srcNode][adjacentNode] > 0:
            connectedToSourceNodes.append(adjacentNode)
    return connectedToSourceNodes

def getTransTo(path, tgtNode):
    connectedTo = getAdjacentNodes(path, curNode)
    
    for adjNode in connectedTo

In [ ]:
""" get thruput from node A to B """
""" don't count bottlenecks? traversing an edge DOES reduce its capacity """
debugThroughput = False

def getAdjacentNodes(path, srcNode):
    connectedToSourceNodes = []
    for adjacentNode in range(len(path)):
        if path[srcNode][adjacentNode] > 0:
            connectedToSourceNodes.append(adjacentNode)
    return connectedToSourceNodes
    
def getThroughputHelper(path, curNode, tgtNode, visitedNodes, maxTransportVals, localVisit, depth):
    connectedNodes = getAdjacentNodes(path, curNode)
    localVisit[curNode] = True
    
    if debugThroughput:
        prntStr = '_._'*depth+'f('+chr(65+curNode)+') = ' 
        prntStr += ' + '.join(['min( ['+chr(65+curNode)+'->'+chr(65+x)+'],f('+chr(65+x)+') )' for x in connectedNodes])
        prntStr += '   [d='+str(depth)+']'
        print(prntStr)
    
    #throughput from curNode to target is sum of each connected/child node's throughput, limited by the amount we can transfer directly to the children (path[cur][child])
    #additionally, once we confirm passing some amount to target via some route, we won't allow a different route to reuse that capacity...
    toTarget = 0
    for adjNode in connectedNodes:
        if debugThroughput: print('  '*depth+chr(65+curNode)+': min( '+str(path[curNode][adjNode])+', f('+chr(65+adjNode)+') )')
        
        if adjNode == tgtNode:
            adjTransport = 99999999
        elif visitedNodes[adjNode]:
            adjTransport = maxTransportVals[adjNode]
        elif localVisit[adjNode]:
            continue
        else:
            adjTransport = getThroughputHelper(path, adjNode, tgtNode, visitedNodes, maxTransportVals, localVisit, depth+1)
        
        toTarget += min(path[curNode][adjNode] , adjTransport)
        
    if debugThroughput: print('  '*depth+'f('+chr(65+curNode)+') = '+str(toTarget))
    maxTransportVals[curNode] = max(maxTransportVals[curNode], toTarget)
    visitedNodes[curNode] = True
    return toTarget


def getThroughput(path, tgtNode):
    N = len(path)
    maxTransportVals = [0] * N
    visitedNodes = [False] * N
    for srcNode in range(N):
        if srcNode != tgtNode:
            localVisit = [False] * N
            getThroughputHelper(path, srcNode, tgtNode, visitedNodes, maxTransportVals, localVisit,0)
    
    return maxTransportVals

In [229]:
p = [
  [0, 7, 4, 6, 0, 0],  # Room 0: Bunnies
  [0, 0, 5, 2, 0, 0],  # Room 1: Bunnies
  [0, 0, 0, 0, 4, 4],  # Room 2: Intermediate room
  [0, 0, 0, 0, 6, 6],  # Room 3: Intermediate room
  [0, 0, 0, 0, 0, 9],  # Room 4: Escape pods
  [0, 0, 0, 0, 0, 0],  # Room 5: Escape pods
]

getTransportTo(p, 5)

[17, 7, 8, 12, 9, 0]

In [219]:
sum(p[4])

45

In [231]:
p = [
 [0, 2, 6, 3, 7, 8, 6, 6],
 [6, 0, 2, 6, 5, 3, 9, 5],
 [6, 4, 0, 7, 1, 9, 4, 1],
 [8, 1, 8, 0, 6, 5, 4, 9],
 [5, 6, 1, 9, 0, 9, 6, 9],
 [3, 3, 3, 5, 1, 0, 8, 1],
 [7, 7, 2, 6, 3, 9, 0, 9],
 [8, 2, 2, 8, 9, 7, 1, 0]
]
getTransportTo(p, len(p)-1)

[38, 36, 32, 41, 45, 24, 43, 0]

In [227]:
# A  B  C  D  E  F  G 
p = [
 [0, 9, 0, 0, 0, 0, 0], # A
 [0, 0, 9, 0, 0, 0, 0], # B
 [0, 0, 0, 3, 6, 0, 0], # C
 [0, 0, 0, 0, 0, 0, 6], # D  #change D-G to 6/9/3 see how doublecounting/not finding bottleneck working
 [0, 0, 0, 6, 0, 6, 0], # E
 [0, 0, 6, 0, 0, 0, 0], # F
 [0, 0, 0, 0, 0, 0, 0], # G
]
getTransportTo(p, len(p)-1)

[9, 9, 9, 6, 6, 6, 0]

In [135]:
chr(65)

'A'